# Guider mode image reduction
This is intended to read the guider mode FITS files and build a movie of the primary segment.\
Craig Lage - 28-Mar-25

In [ ]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath

In [ ]:
# Example of getting a FITS file stored at USDF
# LSSTComCam science CCD
#filename = "s3://rubin-summit/LSSTComCam/20250410/CC_O_20240807_000009/CC_O_20240807_000009_R22_S00_guider.fits"

# LSSTcam guider CCD
filename = "s3://embargo@rubin-summit/LSSTCam/20250410/MC_O_20250410_000007/MC_O_20250410_000007_R00_SG0_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
len(hdu_list)

# Get the main header and the information it contains

In [ ]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    #raft = hdr0['RAFTBAY']
    #ccd = hdr0['CCDSLOT']
    #dayObs = hdr0['DAYOBS']
    #seqNum = hdr0['SEQNUM']
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDER']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRows, roiCols, roiUnder, nStamps, xor]

In [ ]:
# Set the scaling
autoscale = True
# Scale to use if autoscale = False
vmin = 14500
vmax = 14900

# Now build the individual movie frames

In [ ]:
dayObs = 20250410
seqNum = 6
expId = int(f"{dayObs}{seqNum:05d}")
raft = 'R00'
ccd = 'SG0'

filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits" 
rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)

dirName = f"/home/c/cslage/u/Guider_Mode/LSSTCam_movie_{expId}_{raft}_{ccd}"
%mkdir -p {dirName}
movieName = f"Guider_{expId}_{raft}_{ccd}.mp4"
print(movieName)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
# Build the individual frames
fig = plt.figure(figsize=(5,5))
for n in range(1, nStamps+1):
    hduNum = n
    hdrn = hdu_list[hduNum].header
    timestamp = hdrn['STMPTIME']
    image_out = (hdu_list[hduNum].data)
    ax = fig.subplots(1,1)
    plt.suptitle(f"Guider mode {raft} {ccd} {expId}, Frame {n+1}\n{timestamp}", fontsize=12)
    if autoscale:
        im = ax.imshow(image_out, interpolation='nearest', origin='upper', cmap='Greys')
    else:
        im = ax.imshow(image_out, interpolation='nearest', origin='upper', vmin=vmin, vmax=vmax, cmap='Greys')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    #fig.colorbar(im, cax=cax)
    plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    if n % 10 == 0:
        print(f"Finished frame {n}")
print("Done building frames")

# Now make the movie

In [ ]:
print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()

In [ ]:
hdr0 = hdu_list[1].header
for key in hdr0.keys():
    print(key, hdr0[key])